In [ ]:
import requests
import pandas as pd
import time

In [ ]:
URL_API = "https://graphql-gateway.axieinfinity.com/graphql"

headers = {
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'es-ES,es;q=0.9,en;q=0.8',
    'cache-control': 'no-cache',
    'content-length': '1010',
    'content-type': 'application/json',
    'cookie': '_ga=GA1.2.581236636.1631802371; _gid=GA1.2.1060654813.1631802371; cf_clearance=ps3wfb304R6sKsauq5bsfRJh.xrDvMo2TWlLub0Vf2s-1631892833-0-250; _gat_gtag_UA_150383258_1=1',
    'origin': 'https://marketplace.axieinfinity.com',
    'pragma': 'no-cache',
    'referer': 'https://marketplace.axieinfinity.com/',
    'sec-ch-ua': '"Google Chrome";v="93", " Not;A Brand";v="99", "Chromium";v="93"',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
}

In [ ]:
query = """query GetAxieBriefList($auctionType: AuctionType, $criteria: AxieSearchCriteria, $from: Int, $sort: SortBy, $size: Int, $owner: String) {
  axies(auctionType: $auctionType, criteria: $criteria, from: $from, sort: $sort, size: $size, owner: $owner) {
    total
    results {
      ...AxieDetail
      transferHistory(from: $from, size: $size) {
        ...TransferRecords
        __typename
      }
      ethereumTransferHistory(from: $from, size: $size) {
        ...TransferRecords
        __typename
      }   
      __typename
    }
    __typename
  }
}

fragment AxieDetail on Axie {
  id
  image
  class
  chain
  name
  birthDate
  bodyShape
  sireId
  sireClass
  matronId
  matronClass
  stage
  title
  breedCount
  level
  stats {
    ...AxieStats
    __typename
  }
  __typename
}

fragment AxieStats on AxieStats {
  hp
  speed
  skill
  morale
  __typename
}

fragment TransferRecords on TransferRecords {
  total
  results {
    from
    to
    timestamp
    txHash
    withPrice
    __typename
  }
  __typename
}
"""

In [ ]:
payload = {
    "operationName": "GetAxieBriefList",
    "variables": {
        "from": 0,
        "size": 10,
        "sort": "IdAsc",
        "auctionType": "Sale",
        "owner": None,
        "criteria": {
            "region": None,
            "parts": None,
            "bodyShapes": None,
            "classes": None,
            "stages": [
                4
            ],
            "numMystic": None,
            "pureness": None,
            "title": None,
            "breedable": None,
            "breedCount": None,
            "hp": [],
            "skill": [],
            "speed": [],
            "morale": []
        }
    },
    "query": query
}

In [ ]:
TOTAL = requests.post(URL_API, json=payload).json()['data']['axies']['total']

START = 495300
SIZE = 50
NO_REQUESTS = TOTAL // SIZE
CHECKPOINTS = SIZE * 500 # Guarda un Checkpoint cada 5000 requests
col_dtypes = {
    'id':'uint32',
    'sireId':'uint16',
    'matronId':'uint16',
    'birthDate':'uint32',
    'stage':'uint8',
    'breedCount':'uint8',
    'level':'uint8',
    'hp':'uint8',
    'speed':'uint8',
    'skill':'uint8',
    'morale':'uint8',
    'withPrice': 'float32'
}

payload['variables']['size'] = SIZE

df = pd.DataFrame()

for i in range(0, NO_REQUESTS * SIZE, SIZE):
  
  if i % (10 * SIZE ) == 0: print('--> Getting from', i, 'To', i + SIZE);

  payload['variables']['from'] = i
  try:
    time.sleep(4)
    res = requests.post(URL_API, json=payload, headers=headers)
    if res.status_code == 200:
      axies = res.json()['data']['axies']['results']
    else:
      raise 'Invalid code'

  except:
    print('------> Error Batch #', i, '-', i+SIZE)
    axies = []

  transactions = []
  for axie in axies:
    axie = { **axie, **axie.pop('stats')}
    txns = axie.pop('ethereumTransferHistory')['results'] + axie.pop('transferHistory')['results']
    transactions += [ {**axie, **t} for t in txns ]
  

  try:
    transactions = pd.DataFrame(transactions).astype(col_dtypes).drop(columns='__typename')
    transactions.withPrice = transactions.withPrice / 1e18
    df = df.append(transactions)
  except:
    print('------> Empty Batch #', i, '-', i+SIZE)

  if i and ( i + SIZE ) % CHECKPOINTS == 0 and not df.empty:
    print('---------> Saving checkpoint!')
    df.to_csv(f'./dataset/dataset_{ i + SIZE - CHECKPOINTS }_{ i + SIZE }.csv')
    df = pd.DataFrame()

print('---------> Saving last checkpoint!')
df.to_csv(f'./dataset/dataset_last.csv')